In [9]:
import set_lib_path

import geopandas as gpd
import os
import pandas as pd
import shapely

from keplergl import KeplerGl
from src.model.results import load_results
from src.model.street_grid.street_segments import load_street_segments

# Files paths

In [10]:
data_path = '../../../data'
output_path = os.path.join(data_path, 'managed_data_files/mobiele_stad/test_set_LC3_b/output')

streets_path = os.path.join(output_path, 'streets.csv')
results_path = os.path.join(output_path, 'densities.csv')

# Load output results

In [11]:
results = load_results(results_path)

# #temporal filter
# results = results[results['timestamp'].isin(['2020-05-10 22:00:00+00:00','2020-05-10 23:00:00+00:00''2020-05-11 00:00:00+00:00','2020-05-11 01:00:00+00:00','2020-05-11 02:00:00+00:00','2020-05-11 03:00:00+00:00','2020-05-11 04:00:00+00:00','2020-05-11 05:00:00+00:00','2020-05-11 06:00:00+00:00','2020-05-11 08:00:00+00:00','2020-05-11 09:00:00+00:00','2020-05-11 10:00:00+00:00'])]

results

,street_object_id,timestamp,bg_density,bike_density,mot_density,pedestrian_density
0,85577,2020-05-10 22:00:00+00:00,2.634434e-07,5.296673e-02,0.169681,6.858965e-02
1,90671,2020-05-10 22:00:00+00:00,-1.282265e-07,5.542659e-02,0.177026,7.328208e-02
2,97107,2020-05-10 22:00:00+00:00,-4.618523e-07,7.266598e-02,0.181068,7.791602e-02
3,97869,2020-05-10 22:00:00+00:00,-4.703569e-07,5.221988e-02,0.186375,6.502054e-02
4,100545,2020-05-10 22:00:00+00:00,4.392225e-02,2.015786e-01,0.504071,2.362324e-01
...,...,...,...,...,...,...
94535,1152185,2020-05-11 10:00:00+00:00,2.488063e-06,1.857770e-01,0.417705,1.594728e-01
94536,1152186,2020-05-11 10:00:00+00:00,2.843182e-06,1.303337e-01,0.433903,1.470624e-01
94537,1152187,2020-05-11 10:00:00+00:00,-6.873308e-08,-6.873308e-08,0.011428,-6.873308e-08
94538,1152188,2020-05-11 10:00:00+00:00,2.053825e-02,9.322129e-02,0.201520,6.493172e-02


# Densities Mapping with Kepler.gl

In [12]:
streets = pd.read_csv(streets_path)
streets['street_object_geometry'] = streets['street_object_geometry'].apply(lambda x : shapely.wkt.loads(x))
streets_mapping = streets.set_index('street_object_id')['street_object_geometry']

In [13]:
results['geometry'] = results['street_object_id'].map(streets_mapping)

### applying street geometry to data points

In [14]:
gdf = gpd.GeoDataFrame(results)
gdf.head()

,street_object_id,timestamp,bg_density,bike_density,mot_density,pedestrian_density,geometry
0,85577,2020-05-10 22:00:00+00:00,2.634434e-07,0.052967,0.169681,0.068590,"LINESTRING (4.39524 51.21355, 4.39522 51.21356..."
1,90671,2020-05-10 22:00:00+00:00,-1.282265e-07,0.055427,0.177026,0.073282,"LINESTRING (4.39494 51.21391, 4.39486 51.21389..."
2,97107,2020-05-10 22:00:00+00:00,-4.618523e-07,0.072666,0.181068,0.077916,"LINESTRING (4.39593 51.21350, 4.39596 51.21358..."
3,97869,2020-05-10 22:00:00+00:00,-4.703569e-07,0.052220,0.186375,0.065021,"LINESTRING (4.39460 51.21436, 4.39507 51.21429)"
4,100545,2020-05-10 22:00:00+00:00,4.392225e-02,0.201579,0.504071,0.236232,"LINESTRING (4.39580 51.21336, 4.39588 51.21335)"


In [15]:
%run hex_config.py
map = KeplerGl(height=1000, data={"density": gdf}, config=config)
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['density'], 'id': '1i070uf9f…